In [25]:
# 初始化智谱 embedding
from typing import Any, Dict, List, Optional

from langchain_core.embeddings import Embeddings
from langchain_core.utils import get_from_dict_or_env
from pydantic import BaseModel, Field, model_validator
import os
from zhipuai import ZhipuAI

class ZhipuAIEmbeddings(BaseModel, Embeddings):
    """ZhipuAI embedding model integration.

    Setup:

        To use, you should have the ``zhipuai`` python package installed, and the
        environment variable ``ZHIPU_API_KEY`` set with your API KEY.

        More instructions about ZhipuAi Embeddings, you can get it
        from  https://open.bigmodel.cn/dev/api#vector

        .. code-block:: bash

            pip install -U zhipuai
            export ZHIPU_API_KEY="your-api-key"

    Key init args — completion params:
        model: Optional[str]
            Name of ZhipuAI model to use.
        api_key: str
            Automatically inferred from env var `ZHIPU_API_KEY` if not provided.

    See full list of supported init args and their descriptions in the params section.

    Instantiate:

        .. code-block:: python

            from langchain_community.embeddings import ZhipuAIEmbeddings

            embed = ZhipuAIEmbeddings(
                model="embedding-2",
                # api_key="...",
            )

    Embed single text:
        .. code-block:: python

            input_text = "The meaning of life is 42"
            embed.embed_query(input_text)

        .. code-block:: python

            [-0.003832892, 0.049372625, -0.035413884, -0.019301128, 0.0068899863, 0.01248398, -0.022153955, 0.006623926, 0.00778216, 0.009558191, ...]


    Embed multiple text:
        .. code-block:: python

            input_texts = ["This is a test query1.", "This is a test query2."]
            embed.embed_documents(input_texts)

        .. code-block:: python

            [
                [0.0083934665, 0.037985895, -0.06684559, -0.039616987, 0.015481004, -0.023952313, ...],
                [-0.02713102, -0.005470169, 0.032321047, 0.042484466, 0.023290444, 0.02170547, ...]
            ]
    """  # noqa: E501

    client: Any = Field(default=None, exclude=True)  #: :meta private:
    model: str = Field(default="embedding-2")
    """Model name"""
    api_key: str
    """Automatically inferred from env var `ZHIPU_API_KEY` if not provided."""
    dimensions: Optional[int] = None
    """The number of dimensions the resulting output embeddings should have.

    Only supported in `embedding-3` and later models.
    """

    @model_validator(mode="before")
    @classmethod
    def validate_environment(cls, values: Dict) -> Any:
        """Validate that auth token exists in environment."""
        values["api_key"] = get_from_dict_or_env(values, "api_key", "ZHIPUAI_API_KEY")
        try:
            from zhipuai import ZhipuAI

            values["client"] = ZhipuAI(api_key=values["api_key"])
        except ImportError:
            raise ImportError(
                "Could not import zhipuai python package."
                "Please install it with `pip install zhipuai`."
            )
        return values



    def embed_query(self, text: str) -> List[float]:
        """
        Embeds a text using the AutoVOT algorithm.

        Args:
            text: A text to embed.

        Returns:
            Input document's embedded list.
        """
        resp = self.embed_documents([text])
        return resp[0]




    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        Embeds a list of text documents using the AutoVOT algorithm.

        Args:
            texts: A list of text documents to embed.

        Returns:
            A list of embeddings for each document in the input list.
            Each embedding is represented as a list of float values.
        """
        if self.dimensions is not None:
            resp = self.client.embeddings.create(
                model=self.model,
                input=texts,
                dimensions=self.dimensions,
            )
        else:
            resp = self.client.embeddings.create(model=self.model, input=texts)
        embeddings = [r.embedding for r in resp.data]
        return embeddings


In [ ]:
import os
from dotenv import load_dotenv
from pprint import pprint
import chromadb
from langchain.llms import Tongyi
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# 载入环境变量
load_dotenv()

# 1. 定义嵌入函数（使用预训练的嵌入模型）
embedding = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=os.getenv("ZHIPU_API_KEY"),
    dimensions=1024
)

# 2. 初始化 Chroma 向量存储（使用 LangChain 的 Chroma 类）
persist_directory = "ch16_db"  # 与您之前使用的路径保持一致
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
    collection_name="products"
)

# 3. 准备要添加的文档和元数据
documents = ["Galaxy S21", "iPhone 13", "MacBook Pro"]
metadatas = [
    {"category": "手机", "price": 799.99},
    {"category": "手机", "price": 999.99},
    {"category": "笔记本电脑", "price": 1299.99}
]
ids = ["prod1", "prod2", "prod3"]

# 4. 将文档和元数据转换为 LangChain 的 Document 对象
document_objects = [Document(page_content=doc, metadata=meta) for doc, meta in zip(documents, metadatas)]

# 5. 添加文档到 Chroma 向量存储中
vectorstore.add_documents(documents=document_objects, ids=ids)

print("数据添加完成！")

# 6. 获取集合中的所有数据
all_data = vectorstore._collection.get()
print("集合中的所有数据：")
pprint(all_data)

# 7. 根据 ID 获取特定的文档
specific_data = vectorstore._collection.get(ids=["prod1"])
print("\nID 为 'prod1' 的文档：")
pprint(specific_data)

# 8. 根据元数据条件获取文档
filtered_data = vectorstore._collection.get(where={"category": "手机"})
print("\n类别为 '手机' 的文档：")
pprint(filtered_data)

# 9. 更新已有文档的元数据
vectorstore._collection.update(
    ids=["prod1"],
    metadatas=[{"category": "手机", "price": 749.99}]
)
print("\n已更新 ID 为 'prod1' 的文档价格。")

# 10. 删除特定 ID 的文档
vectorstore._collection.delete(ids=["prod2"])
print("\n已删除 ID 为 'prod2' 的文档。")

# 11. 查看集合中剩余的文档
remaining_data = vectorstore._collection.get()
print("\n剩余的文档：")
pprint(remaining_data)

# 12. 初始化 Tongyi 语言模型
llm = Tongyi(
    model='qwen-plus',  # 替换为您使用的 Tongyi 模型名称
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 13. 创建检索器（Retriever）
retriever = vectorstore.as_retriever()

# 14. 创建 RetrievalQA 链
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" 是一种简单的链类型，您可以根据需要选择其他类型
    retriever=retriever,
    return_source_documents=True
)

# 15. 执行简单查询（方法一）
query = "苹果手机"
result = qa({"query": query})
print("\n简单查询结果：")
print(result["result"])

# 如果您不需要来源文档，可以在创建 qa 对象时将 return_source_documents 设置为 False：
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False
)
# 然后可以使用 qa.run(query) 方法：
result = qa.run(query)
print("\n简单查询结果：")
print(result)

# 16. 执行带来源的查询
result_with_sources = qa({"query": query})
print("\n带来源的查询结果：")
pprint(result_with_sources)

In [ ]:
import os
from dotenv import load_dotenv
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import Tongyi  # 或者其他语言模型，例如 Tongyi

# 加载环境变量
load_dotenv()


# 1. 定义嵌入函数（使用预训练的嵌入模型）
embedding = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=os.getenv("ZHIPU_API_KEY"),
    dimensions=1024
)

# 2. 初始化 Chroma 向量存储（使用 LangChain 的 Chroma 类）
persist_directory = "ch16_db"  # 与您之前使用的路径保持一致
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
    collection_name="products"
)
# 1. 加载文档
loader = TextLoader('/root/autodl-tmp/lizhenping/langchain_tutorial/LangChain_CookBook/data/市场分析报告.txt', encoding='utf-8')
documents = loader.load()

# 2. 创建向量存储索引
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=embedding,  # 替换为您使用的嵌入模型实例
)
index = index_creator.from_loaders([loader])

# 3. 执行简单查询
query = "这篇文章的主要内容是什么？"
response = index.query(query)
print("简单查询结果：")
print(response)


llm = Tongyi(
    model='qwen-plus',  # 替换为您使用的 Tongyi 模型名称
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

retriever = index.vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 您可以根据需要选择其他链类型
    retriever=retriever
)

# 5. 执行高级查询
advanced_query = "请详细总结一下文章的关键观点。"
result = qa_chain.run(advanced_query)
print("高级查询结果：")
print(result)

In [ ]:

import os
from dotenv import load_dotenv
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import Tongyi  # 或者其他语言模型，例如 Tongyi

# 加载环境变量
load_dotenv()


# 1. 定义嵌入函数（使用预训练的嵌入模型）
embedding = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=os.getenv("ZHIPU_API_KEY"),
    dimensions=1024
)

# 2. 初始化 Chroma 向量存储（使用 LangChain 的 Chroma 类）
persist_directory = "ch16_db"  # 与您之前使用的路径保持一致
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
    collection_name="products"
)
# 1. 加载文档
loader = TextLoader('/root/autodl-tmp/lizhenping/langchain_tutorial/LangChain_CookBook/data/市场分析报告.txt', encoding='utf-8')
documents = loader.load()

# 2. 创建向量存储索引
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=embedding,  # 替换为您使用的嵌入模型实例
)
index = index_creator.from_loaders([loader])

# 3. 初始化语言模型
llm = Tongyi(
    model='qwen-plus',  # 替换为您使用的 Tongyi 模型名称
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)
# 4. 执行简单查询，提供 llm 参数
query = "这篇文章的主要内容是什么？"
response = index.query(query, llm=llm)
print("简单查询结果：")
print(response)

# 5. 创建一个检索QA链
retriever = index.vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 您可以根据需要选择其他链类型
    retriever=retriever
)

# 6. 执行高级查询
advanced_query = "请详细总结一下文章的关键观点。"
result = qa_chain.run(advanced_query)
print("高级查询结果：")
print(result)



In [ ]:

import os
from dotenv import load_dotenv
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import Tongyi  # 或者其他语言模型，例如 Tongyi

# 加载环境变量
load_dotenv()

# 1. 定义嵌入函数（使用预训练的嵌入模型）
embedding = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=os.getenv("ZHIPU_API_KEY"),
    dimensions=1024
)

# 2. 初始化 Chroma 向量存储（使用 LangChain 的 Chroma 类）
persist_directory = "ch16_db"  # 与您之前使用的路径保持一致
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
    collection_name="products"
)
# 1. 加载文档
loader = TextLoader('/root/autodl-tmp/lizhenping/langchain_tutorial/LangChain_CookBook/data/市场分析报告.txt', encoding='utf-8')
documents = loader.load()

# 2. 创建向量存储索引
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=embedding,  # 替换为您使用的嵌入模型实例
)
index = index_creator.from_loaders([loader])

# 3. 获取底层的向量存储（Vectorstore）
vectorstore = index.vectorstore

# 4. 查询 Vectorstore 中的向量信息
# 获取所有向量的 IDs
all_ids = vectorstore._collection.get()['ids']
print("所有向量的 IDs：")
print(all_ids)

# 5. 根据 ID 获取向量和元数据
result = vectorstore._collection.get(ids=[all_ids[0]])
print("\n获取指定 ID 的向量和元数据：")
print("IDs:", result['ids'])
print("Embeddings:", result['embeddings'])
print("Metadatas:", result['metadatas'])
print("Documents:", result['documents'])

# 6. 更新向量的元数据
new_metadata = {"source": "updated_source.txt"}
vectorstore._collection.update(ids=[all_ids[0]], metadatas=[new_metadata])
print("\n更新后的元数据：")
updated_result = vectorstore._collection.get(ids=[all_ids[0]])
print("Metadatas:", updated_result['metadatas'])

# 7. 删除指定的向量
vectorstore._collection.delete(ids=[all_ids[0]])
print("\n删除后的所有向量 IDs：")
print(vectorstore._collection.get()['ids'])

# 8. 添加新的向量
new_document = "这是一个新的文档内容。"
new_embedding = vectorstore.embedding_function.embed_documents([new_document])[0]
new_id = "new_doc_id"
new_metadata = {"source": "new_document.txt"}
vectorstore._collection.add(
    ids=[new_id],
    embeddings=[new_embedding],
    documents=[new_document],
    metadatas=[new_metadata]
)
print("\n添加新向量后的所有 IDs：")
print(vectorstore._collection.get()['ids'])

# 9. 执行相似性搜索
query = "查询内容"
query_embedding = vectorstore.embedding_function.embed_query(query)
search_results = vectorstore._collection.query(
    query_embeddings=[query_embedding],
    n_results=2  # 返回最相似的 2 个结果
)
print("\n相似性搜索结果：")
print("IDs:", search_results['ids'])
print("Scores:", search_results['distances'])
print("Documents:", search_results['documents'])

# 10. 使用 LLM 生成答案
# 3. 初始化语言模型
llm = Tongyi(
    model='qwen-plus',  # 替换为您使用的 Tongyi 模型名称
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 将搜索结果的文档内容合并为上下文
context = "\n".join(search_results['documents'][0])

# 构建提示（Prompt）
prompt = f"基于以下内容回答问题：\n{context}\n\n问题：{query}\n\n答案："

# 使用 LLM 生成答案
answer = llm(prompt)
print("\nLLM 生成的答案：")
print(answer)



In [ ]:

import os
from dotenv import load_dotenv
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.llms import Tongyi  # 使用通义的语言模型

# 加载环境变量
load_dotenv()

# 1. 初始化嵌入模型
embedding = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key=os.getenv("ZHIPU_API_KEY"),
    dimensions=1024
)

# 2. 加载文档
document_path = '/root/autodl-tmp/lizhenping/langchain_tutorial/LangChain_CookBook/data/市场分析报告.txt'
loader = TextLoader(document_path, encoding='utf-8')
documents = loader.load()

# 3. 创建向量存储索引
persist_directory = "ch16_db"  # 指定向量存储的持久化目录
collection_name = "products"   # 指定集合名称

vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
    collection_name=collection_name
)

# 如果需要重新创建索引，可以先清空现有的集合
# vectorstore._collection.delete()

# 使用 VectorstoreIndexCreator 创建索引并添加到向量存储中
# 2. 创建向量存储索引
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=embedding,  # 替换为您使用的嵌入模型实例
)
index = index_creator.from_loaders([loader])

# 4. 获取底层的向量存储对象
vectorstore = index.vectorstore

# 5. 查询 Vectorstore 中的向量信息
# 获取所有向量的 IDs
all_ids = vectorstore._collection.get()['ids']
print("所有向量的 IDs：")
print(all_ids)

# 6. 根据 ID 获取向量和元数据（Read）
result = vectorstore._collection.get(ids=[all_ids[0]])
print("\n获取指定 ID 的向量和元数据：")
print("IDs:", result['ids'])
print("Embeddings:", result['embeddings'])
print("Metadatas:", result['metadatas'])
print("Documents:", result['documents'])

# 7. 更新向量的元数据（Update）
new_metadata = {"source": "updated_source.txt"}
vectorstore._collection.update(ids=[all_ids[0]], metadatas=[new_metadata])
print("\n更新后的元数据：")
updated_result = vectorstore._collection.get(ids=[all_ids[0]])
print("Metadatas:", updated_result['metadatas'])

# 8. 添加新的向量（Create）
new_document = "这是一个新的文档内容，用于测试添加操作。"
new_embedding = embedding.embed_documents([new_document])[0]
new_id = "new_doc_id"
new_metadata = {"source": "new_document.txt"}
vectorstore._collection.add(
    ids=[new_id],
    embeddings=[new_embedding],
    documents=[new_document],
    metadatas=[new_metadata]
)
print("\n添加新向量后的所有 IDs：")
print(vectorstore._collection.get()['ids'])

# 9. 删除指定的向量（Delete）
vectorstore._collection.delete(ids=[all_ids[0]])
print("\n删除后的所有向量 IDs：")
print(vectorstore._collection.get()['ids'])

# 10. 执行相似性搜索
query = "请提供市场分析的摘要。"
query_embedding = embedding.embed_query(query)
search_results = vectorstore._collection.query(
    query_embeddings=[query_embedding],
    n_results=2  # 返回最相似的 2 个结果
)
print("\n相似性搜索结果：")
print("IDs:", search_results['ids'])
print("Scores:", search_results['distances'])
print("Documents:", search_results['documents'])

# 11. 使用 LLM 生成答案
llm = Tongyi(
    model='qwen-plus',  # 替换为您使用的 Tongyi 模型名称
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 将搜索结果的文档内容合并为上下文
context = "\n".join(search_results['documents'][0])

# 构建提示（Prompt）
prompt = f"基于以下内容回答问题：\n{context}\n\n问题：{query}\n\n答案："

# 使用 LLM 生成答案
answer = llm(prompt)
print("\nLLM 生成的答案：")
print(answer)
